In [38]:
from dataset.threedfront_dataset import ThreedFrontDatasetSceneGraph

In [39]:
import numpy as np

In [40]:
dataset = ThreedFrontDatasetSceneGraph(
    root="/home/ubuntu/datasets/FRONT",
    split='val_scans',
    shuffle_objs=True,
    use_SDF=False,
    use_scene_rels=True,
    with_changes=True,
    with_CLIP=True,
    large=False,
    seed=False,
    room_type='all',
    recompute_clip=False)



loading CLIP
Checking for missing clip feats. This can be slow the first time.


100%|██████████| 370/370 [00:00<00:00, 418.62it/s]


In [76]:
a = dataset[0]
for ew, dw in zip(a['encoder']['words'], a['decoder']['words']):
    print(ew)


lamp left cabinet
nightstand same super category as wardrobe
cabinet same super category as nightstand
bed taller than table
wardrobe same material as nightstand
nightstand close by bed
bed taller than chair
lamp above table
table left cabinet
chair right nightstand
bed front table
table left lamp
wardrobe front lamp
nightstand smaller than cabinet
cabinet taller than nightstand
nightstand shorter than table
lamp above chair
wardrobe taller than cabinet
wardrobe bigger than table
bed left cabinet
table bigger than nightstand
wardrobe taller than chair
table same material as chair
table close by bed
lamp bigger than chair
cabinet behind lamp
nightstand front chair
bed standing on floor
wardrobe front chair
cabinet same super category as wardrobe
bed bigger than lamp
chair shorter than nightstand
lamp above floor
cabinet shorter than bed
chair close by table
chair shorter than table
nightstand left cabinet
wardrobe bigger than nightstand
lamp above bed
wardrobe bigger than chair
chair be

In [77]:
txt_list = []
for x in ['encoder']:
    en_obj = a[x]['objs'].cpu().numpy().astype(np.int32)
    en_triples = a[x]['triples'].cpu().numpy().astype(np.int32)
    # print(en_triples)
    #instance
    sub = en_triples[:,0]
    obj = en_triples[:,2]
    #cat
    instance_ids = np.array(sorted(list(set(sub.tolist() + obj.tolist())))) #0-n
    cat_ids = en_obj[instance_ids]
    texts = [dataset.classes_r[cat_id] for cat_id in cat_ids]
    objs = dict(zip(instance_ids.tolist(),texts))
    objs = {str(key): value for key, value in objs.items()}
    print(objs)
    for rel in en_triples[:,1]:
        if rel == 0:
            txt = 'in'
            txt_list.append(txt)
            continue
        txt = dataset.relationships_dict_r[rel]
        txt_list.append(txt)
txt_list = np.array(txt_list)
print(txt)
rel_list = np.vstack((sub,obj,en_triples[:,1],txt_list)).transpose()
print(a['scan_id'])
print(rel_list)

{'0': 'lamp', '1': 'floor', '2': 'wardrobe', '3': 'bed', '4': 'cabinet', '5': 'table', '6': 'chair', '7': 'nightstand', '8': '_scene_'}
in
MasterBedroom-33296
[['0' '4' '1' 'left']
 ['7' '2' '14' 'same super category as']
 ['4' '7' '14' 'same super category as']
 ['3' '5' '10' 'taller than']
 ['2' '7' '15' 'same material as']
 ['7' '3' '5' 'close by']
 ['3' '6' '10' 'taller than']
 ['0' '5' '6' 'above']
 ['5' '4' '1' 'left']
 ['6' '7' '2' 'right']
 ['3' '5' '3' 'front']
 ['5' '0' '1' 'left']
 ['2' '0' '3' 'front']
 ['7' '4' '9' 'smaller than']
 ['4' '7' '10' 'taller than']
 ['7' '5' '11' 'shorter than']
 ['0' '6' '6' 'above']
 ['2' '4' '10' 'taller than']
 ['2' '5' '8' 'bigger than']
 ['3' '4' '1' 'left']
 ['5' '7' '8' 'bigger than']
 ['2' '6' '10' 'taller than']
 ['5' '6' '15' 'same material as']
 ['5' '3' '5' 'close by']
 ['0' '6' '8' 'bigger than']
 ['4' '0' '4' 'behind']
 ['7' '6' '3' 'front']
 ['3' '1' '7' 'standing on']
 ['2' '6' '3' 'front']
 ['4' '2' '14' 'same super category a

In [53]:
sub.shape, obj.shape, en_triples.shape, txt_list.shape

((77,), (77,), (77, 3), (77,))